In [ ]:
#!pip install sentence-transformers

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = '/Users/patrickokwir/Desktop/Git_Projects/Ted-Talks-Recommender-System/Data_output/ted_talk_clean_merged_bert.csv'
df = pd.read_csv(data, index_col=0)
df.head()

,author,title,description,likes,views,transcript,date,tags
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",Jun 2023,"art, creativity, design, fashion, performance"
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,Jun 2023,"diversity, identity, inclusion, indigenous_peo..."
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,Jun 2023,"business, community, equality, humanity, money..."
3,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...",Jun 2023,"africa, animation, education, history, identit..."
4,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...,Jun 2023,"biotech, brain, disability, health, invention,..."


In [3]:
# new column 'test' to df contact strings in title

df['test'] = df['title'] + 'Tags: ' + df['tags']
df.head()

,author,title,description,likes,views,transcript,date,tags,test
0,Machine Dazzle,how to unleash your inner maximalist through c...,tapping into the transformational power of cos...,8100,270192,"Hello, I am Machine Dazzle, and I am an emotio...",Jun 2023,"art, creativity, design, fashion, performance",how to unleash your inner maximalist through c...
1,Jioji Ravulo,a liberating vision of identity that transcend...,how can we move past societys inclination to b...,9200,309952,Can you paint with all the colors of the wind?...,Jun 2023,"diversity, identity, inclusion, indigenous_peo...",a liberating vision of identity that transcend...
2,Rebecca Darwent,how to fund real change in your community,is there a way to give back that benefits ever...,1000,341218,I spent my whole career in the nonprofit secto...,Jun 2023,"business, community, equality, humanity, money...",how to fund real change in your communityTags:...
3,Susanne Buckley-Zistel,what caused the rwandan genocide,for one hundred days in 1994 the african count...,3700,126376,"For 100 days in 1994, the African country of R...",Jun 2023,"africa, animation, education, history, identit...","what caused the rwandan genocideTags: africa, ..."
4,Conor Russomanno,a powerful new neurotech tool for augmenting y...,in an astonishing talk and tech demo neurotech...,1100,374259,I became obsessed with the relationship betwee...,Jun 2023,"biotech, brain, disability, health, invention,...",a powerful new neurotech tool for augmenting y...


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-MiniLM-L6-v2')

In [ ]:
tokens = {'input_ids': [], 'attention_mask': []}

In [ ]:
df_rows = df.index.tolist()
df_rows = df_rows[0:300]

In [ ]:
for row in df_rows:
    new_tokens = tokenizer.encode_plus(df['test'][row],
                                  max_length=512,
                                  truncation=True,
                                  padding='max_length',
                                  return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

In [ ]:
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [ ]:
tokens['input_ids'].shape, tokens['attention_mask'].shape

In [ ]:
outputs = model(**tokens)
outputs.keys()

In [ ]:
embeddings = outputs.last_hidden_state
embeddings.shape

In [ ]:
#mean pooling
mask = tokens['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()


In [ ]:
mask_embeddings = embeddings * mask

In [ ]:
embeddings.shape

In [ ]:
mask_embeddings.shape

In [ ]:
summed = torch.sum(mask_embeddings, dim=1)
summed.shape

In [ ]:
counts = torch.clamp(mask.sum(dim=1), min=1e-9)
counts.shape

In [ ]:
#Document embeddings are now the mean of the word embeddings
mean_pooled = summed/counts
mean_pooled.shape

In [ ]:
mean_pooled

In [ ]:
# get cosine simmilarity
from sklearn.metrics.pairwise import cosine_similarity # this expects a numpy array

In [ ]:
mean_pooled = mean_pooled.detach().numpy() # detach to avoid gradient accumulation, also code will not work if you don't detach

#get cosine similarities
cosine_similarity(
                    [mean_pooled[2]], 
                    mean_pooled[1:]
                )

In [ ]:
similar_talks = cosine_similarity(
                    [mean_pooled[22]], 
                    mean_pooled[1:]
                )
similar_talks = np.argsort(similar_talks)[::-1][:1
                ]
print(similar_talks)

In [ ]:
index = df[df['title'] == 'how to fund real change in your community'].index[0]
print(index)

In [ ]:
df.iloc[22]['test']

In [ ]:
#return title of talk in index 108 of df
df.iloc[270]['test']

In [ ]:
# define a function to get the top n similar talks, takes the title and the number of similar talks
# mean_pooled = mean_pooled.detach().numpy() 
def get_similar_talks(title, n):
    # get the index of the talk in df using the title
    index = df[df['title'] == title].index[0]
    # get the list of similar talks
    similar_talks = cosine_similarity([mean_pooled[index]], mean_pooled)
    # sort the list of similar talks
    sorted_similar_talks = np.argsort(similar_talks[0])[::-1]
    # get the top n similar talks
    top_n_similar_talks = sorted_similar_talks[0:n]
    # get the titles of the top n similar talks
    top_n_similar_talks_titles = [df['title'][i] for i in top_n_similar_talks]
    return top_n_similar_talks_titles



In [ ]:
get_similar_talks(title = 'how to fund real change in your community', n=5)

In [5]:
from transformers import pipeline

In [7]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ba63a641-bbf8-4fca-b3ae-7a26041fb0ea)')' thrown while requesting HEAD https://huggingface.co/facebook/bart-large-cnn/resolve/main/config.json
